In [52]:
import pandas as pd
import numpy as np

raw_data=pd.read_excel("rawData.xlsx")
print(raw_data.info())
print(raw_data)

df=raw_data.dropna(axis=1,how='all')
print(df.columns)
df=df.drop(columns=['ASA','镇痛药诱导剂量','镇痛追加次数','镇痛药追加剂量','镇静追加4','镇静追加3','镇静追加2','镇静药追加剂量','有无追加镇静','镇静药诱导剂量'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245 entries, 0 to 1244
Columns: 185 entries, 受试者编号 to 术后24内患者的满意度评价非常不满意
dtypes: datetime64[ns](2), float64(111), int64(15), object(57)
memory usage: 1.8+ MB
None
      受试者编号     研究开始时间     研究结束时间 性别    年龄   身高    体重 ASA 有无手术史      手术说明  \
0         1 2020-11-26 2020-11-26  女  72.0  158  50.0  Ⅱ级     无       NaN   
1         2 2020-11-26 2020-11-26  女  61.0  159  70.0  Ⅱ级     无       NaN   
2         3 2020-11-27 2020-11-27  男  58.0  169  83.0  Ⅱ级     无       NaN   
3         4 2020-11-27 2020-11-27  女  65.0  154  60.0  Ⅱ级     无       NaN   
4         5 2020-11-27 2020-11-27  女  55.0  160  60.0  Ⅱ级     无       NaN   
...     ...        ...        ... ..   ...  ...   ...  ..   ...       ...   
1240   1242 2021-01-30 2021-01-30  女  37.0  155  46.0  Ⅱ级     有    结肠CA术后   
1241   1243 2021-01-30 2021-01-30  女  67.0  152  54.0  Ⅱ级     无       NaN   
1242   1244 2021-01-30 2021-01-30  女  51.0  152  52.0  Ⅱ级     有  胃良性肿瘤切除术   
1243   1245 2021

In [53]:
mean={}
var={}

for i in ['身高','体重','年龄']:
    mean[i] = df[i].mean()
    var[i]=df[i].var()


##最后计算
def process_stn(row):
    row['身高']=(row['身高']-mean['身高'])/var['身高']
    row['体重']=(row['体重']-mean['体重'])/var['体重']
    row['年龄']=(row['年龄']-mean['年龄'])/var['年龄']
    return row
def process_stn2(row):
    row['身高']=row['身高']/200
    row['体重']=row['体重']/120
    row['年龄']=row['年龄']/100
    return row


def process_map01(df):
    df['性别']=df['性别'].replace({'男':1,'女':0})
    df['有无手术史']=df['有无手术史'].replace({'有':1,'无':0})
    df['有无既往史']=df['有无既往史'].replace({'有':1,'无':0})
    df['是否吸烟']=df['是否吸烟'].replace({'从未吸烟者':0,'经常吸烟者：每天吸卷烟1支以上，连续或累计6个月':1,'偶尔吸烟者：每周吸卷烟超过4次，但平均每天不足1支':2})
    df['是否酗酒']=df['是否酗酒'].replace({'有':1,'无':0})
    df['有无PONV']=df['有无PONV'].replace({'有':1,'无':0})
    df['有无晕动史']=df['有无晕动史'].replace({'有':1,'无':0})
    df['麻醉医生满意度']=df['麻醉医生满意度'].str.extract(r'(\d+)').astype(float)
    df['内镜医生满意度']=df['内镜医生满意度'].str.extract(r'(\d+)').astype(float)

    column_toreplace= df.columns[df.columns.str.contains('如果有，那么是出现了')]
    column_toreplace2= df.columns[df.columns.str.contains('有没')]
    column_toreplace3= df.columns[df.columns.str.contains('是否出现')]
    # column_toreplace=column_toreplace.(column_toreplace2)
    column_toreplace= column_toreplace.append([column_toreplace2,column_toreplace3])
    print(column_toreplace)
    df[column_toreplace]= df[column_toreplace].fillna(0)
    df['镇静追加次数']=df['镇静追加次数'].fillna(0)
    df[column_toreplace]=df[column_toreplace].replace({'是':1})
    # print(df[column_toreplace])
    # print(df['开始给药时间'])
    return df
def process_row(row):
    for column,value in row.items():
        if column=='术后24内患者的满意度评价非常满意'and not pd.isna(value):row['整体满意度']=4
        elif column=='术后24内患者的满意度评价满意'and not pd.isna(value):row['整体满意度']=3
        elif column=='术后24内患者的满意度评价一般'and not pd.isna(value):row['整体满意度']=2
        elif column=='术后24内患者的满意度评价不满意'and not pd.isna(value):row['整体满意度']=1
        elif column=='术后24内患者的满意度评价非常不满意'and not pd.isna(value):row['整体满意度']=0
    if row['呛咳']=='无' and row['体动']=='无' and row['术中其他']=='无':row['术中不良反应种类']=0
    elif row['呛咳']=='有' and row['体动']=='无' and row['术中其他']=='无':row['术中不良反应种类']=1
    elif row['呛咳']=='有' and row['体动']=='有' and row['术中其他']=='无':row['术中不良反应种类']=2
    elif row['呛咳']=='有' and row['体动']=='有' and row['术中其他']=='有':row['术中不良反应种类']=3
    elif row['呛咳']=='有' and row['体动']=='无' and row['术中其他']=='有':row['术中不良反应种类']=4
    elif row['呛咳']=='无' and row['体动']=='有' and row['术中其他']=='无':row['术中不良反应种类']=5
    elif row['呛咳']=='无' and row['体动']=='有' and row['术中其他']=='有':row['术中不良反应种类']=6
    elif row['呛咳']=='无' and row['体动']=='无' and row['术中其他']=='有':row['术中不良反应种类']=7
    row['镇静药用药量']=row['镇静药总剂量']/row['体重']
    row['镇痛药用药量']=row['镇痛药总剂量']/row['体重']
    # print(type(row['开始给药时间']))
    # row['开始给药时间']=pd.to_timedelta(row['开始给药时间'])
    # row['最后一次给药时间']=pd.to_timedelta(row['最后一次给药时间'])
    # row['IPI达到4分时间']=pd.to_timedelta(row['IPI达到4分时间'])
    # row['睁眼时间']=pd.to_timedelta(row['睁眼时间'])
    # row['给药的间隔时间']=(row['最后一次给药时间']-row['开始给药时间']).astype('timedelta64[m]')
    # row['沉睡的间隔时间']=(row['睁眼时间']-row['最后一次给药时间']).astype('timedelta64[m]')
    # row['IPI达到4分的间隔时间']=(row['IPI达到4分时间']-row['最后一次给药时间']).astype('timedelta64[m]')
    return row

# print(df['麻醉医生满意度'])
df_res=process_map01(df)
df_res=df_res.apply(process_row,axis=1)
# print(df_res['麻醉医生满意度'])
df_res=df_res.apply(process_stn,axis=1)
df_res=df_res.drop(['呛咳','体动','术中其他','镇静药总剂量','镇痛药总剂量','术后24内患者的满意度评价非常满意','术后24内患者的满意度评价满意','术后24内患者的满意度评价一般','术后24内患者的满意度评价不满意','术后24内患者的满意度评价非常不满意','研究开始时间','研究结束时间','术中其他程度'],axis=1)




# print(df_res.info)
# print(df_res['IPI达到4分的间隔时间'])
# df_res.to_excel("t2.xlsx")

Index(['如果有，那么是出现了恶恶心', '如果有，那么是出现了恶呕吐', '如果有，那么是出现了恶都有', '如果有，那么是出现了恶呕吐次数',
       '如果有，那么是出现了头头晕', '如果有，那么是出现了头头昏', '如果有，那么是出现了头头痛', '如果有，那么是出现了头都有',
       '如果有，那么是出现了嗜嗜睡', '如果有，那么是出现了嗜乏力', '如果有，那么是出现了嗜都有', '如果有，那么是出现了腹腹胀',
       '如果有，那么是出现了腹腹痛', '如果有，那么是出现了腹都有', '有没出现嗜睡乏力的情况呢有', '有没出现嗜睡乏力的情况呢无',
       '有没出现腹胀腹痛的情况呢有', '有没出现腹胀腹痛的情况呢无', '还有没其他不舒服的情况呢有', '还有没其他不舒服的情况呢无',
       '是否出现了恶心呕吐的情况是', '是否出现了恶心呕吐的情况否', '是否出现了头晕头昏头痛是', '是否出现了头晕头昏头痛否'],
      dtype='object')


In [54]:
df_res.to_excel("整体excel01.xlsx", index=True)

grouped=df_res.groupby(['镇静药名称'])
for group_name, group_data in grouped:
     group_df = pd.DataFrame(group_data)
     group_df.to_excel(f"group_{group_name}01.xlsx", index=True)

C:\Users\16279\AppData\Local\Temp\ipykernel_23484\2763275919.py:4: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group_name, group_data in grouped:
